# Calculate potential Evapo-transpiration (PET) girds
The purpose of this notebook is to create PET grids as part of the Cambodia/eWater project. The notebook uses daily min,mean and max air temperature grids and the Hargreaves equation.

## Import Libraries

In [1]:
import numpy as np
import math as mt
from datetime import *
import calendar
import netCDF4 as nc
import xarray as xr
import glob

## Create Functions

In [3]:
#Create function to calculate date from netcdf (as units of netcdf are "seconds since 20xx-xx-xx 00:00:00" )
def to_date(epoch, time_unit):
    time_unit =  datetime.strptime(time_unit, '%Y-%m-%d %H:%M:%S') 
    return time_unit + timedelta(seconds=epoch)

#Create function to calulate julian day of year (days since start of year)
def julian_days(epoch, time_unit):
    start_of_year = datetime.strptime('01/01/'+ str(time_unit),'%m/%d/%Y')
    start_of_month = datetime.strptime(min_xr.time.units[14:], '%Y-%m-%d %H:%M:%S') 
    epoch = start_of_month + timedelta(seconds=epoch)
    j_days = int((((epoch - start_of_year)+timedelta(days=1))/timedelta(days=1)))
    return (j_days)

#Create function to test whether the year of interest is a leap year
def leap_year(year):
    if calendar.isleap(int(year)) == True:
#         print(str(year) +' is a leap year')
        ndy = 366
    else:
#         print(str(year) +' is not a leap year')
        ndy = 365
    return(ndy)

## loop through all temperature grids and save as netCDF format

In [4]:
input_folder = '/g/data/u46/users/ext547/ewater/input_data/Temperature/degrees_celcius_and_clipped/celcius/' #location of files
ncfiles = glob.glob(input_folder + 'original/*.nc') #a glob file containing the names of all nc

for ncpath in ncfiles: #loop through all nc
    #Set dataset variables
    nc_name = ncpath[-28:]
    print ('Processing '+str(nc_name)+' now')
    min_nc = 'min_'+str(nc_name)
    max_nc = 'max_'+str(nc_name)
    mean_nc = 'mean_'+str(nc_name)
    
    #Input temperature datasets
    min_xr = xr.open_dataset(input_folder+'/min/'+min_nc, decode_times=False)
    mean_xr = xr.open_dataset(input_folder+'/mean/'+mean_nc, decode_times=False)
    max_xr = xr.open_dataset(input_folder+'/max/'+max_nc, decode_times=False)
    
    #Calculate date from netcdf and save as xarray dataset
    time = xr.DataArray([to_date(epoch, min_xr.time.units[14:]) for epoch in min_xr.time.values.tolist()], coords={'time': min_xr.time}, dims=('time',))
   
    #Calculate julian days and save as an array
    j_days = xr.DataArray([julian_days(epoch, 
                                       min_xr.time.units[14:18]) for epoch in time.time.values.tolist()], 
                          coords={'time': min_xr.time}, dims=('time',))
    
    #Use leap year function and save the number of days within the year as a variable
    ndy = leap_year( min_xr.time.units[14:18])
    
    #Create array of temperature difference
    #Unit: T_diff in degrees celcius
    T_diff = max_xr.Tair - min_xr.Tair
    
    #Create array of latitude in radians
    #Unit: lat_rad in radians
    lat_rad = np.radians(min_xr.lat)
    
    # Caclulate the inverse relative distance between the Earth and the Sun
    # Unit: dr in radians
    dr = 1 + 0.033 * np.cos(2 * np.pi * j_days / ndy)
    
    # Calculate solar declination
    # Unit: sd in radians
    sd = 0.409 * np.sin(2 * mt.pi * j_days / ndy - 1.39)

    # Calculate a sunset angle input variable
    # Unit: no unit
    sa_X = 1 - (np.tan(lat_rad))**2 * ((np.tan(sd))**2)
    
    # Calculate sunset angle
    # Units: sa in radians
    sa = mt.pi / 2 - np.arctan(-np.tan(lat_rad) * np.tan(sd) / np.sqrt(sa_X))

    # Calcualte extraterrestrial radiation
    # Units: ra in MJ / (m^ day)
    ra = 37.586 * dr * (sa * np.sin(lat_rad) * np.sin(sd) + np.cos(lat_rad) * np.cos(sd) * np.sin(sa))
    
    # Calculate Potential Evapotranspiration
    # Units: PET in mm/day
    pet = 0.0009384 * (mean_xr.Tair + 17.8) * np.sqrt(T_diff) * ra
    pet = xr.Dataset({'PET':pet})
    pet.PET.attrs['units'] = 'mm/day' #add unit to xarray
    
    #Save PET output as netcdf
    pet.to_netcdf(path = '/g/data/u46/users/ext547/ewater/input_data/PET/PET'+nc_name[12:])
    print ('PET'+str(nc_name[12:])+' is complete')

Processing celsius_Tair_WFDEI_198902.nc now
PET_WFDEI_198902.nc is complete
Processing celsius_Tair_WFDEI_199102.nc now
PET_WFDEI_199102.nc is complete
Processing celsius_Tair_WFDEI_198310.nc now
PET_WFDEI_198310.nc is complete
Processing celsius_Tair_WFDEI_200904.nc now
PET_WFDEI_200904.nc is complete
Processing celsius_Tair_WFDEI_199609.nc now
PET_WFDEI_199609.nc is complete
Processing celsius_Tair_WFDEI_201608.nc now
PET_WFDEI_201608.nc is complete
Processing celsius_Tair_WFDEI_199207.nc now
PET_WFDEI_199207.nc is complete
Processing celsius_Tair_WFDEI_198012.nc now
PET_WFDEI_198012.nc is complete
Processing celsius_Tair_WFDEI_201205.nc now
PET_WFDEI_201205.nc is complete
Processing celsius_Tair_WFDEI_198408.nc now
PET_WFDEI_198408.nc is complete
Processing celsius_Tair_WFDEI_199803.nc now
PET_WFDEI_199803.nc is complete
Processing celsius_Tair_WFDEI_200212.nc now
PET_WFDEI_200212.nc is complete
Processing celsius_Tair_WFDEI_201009.nc now
PET_WFDEI_201009.nc is complete
Processing c

PET_WFDEI_199812.nc is complete
Processing celsius_Tair_WFDEI_198008.nc now
PET_WFDEI_198008.nc is complete
Processing celsius_Tair_WFDEI_200102.nc now
PET_WFDEI_200102.nc is complete
Processing celsius_Tair_WFDEI_200002.nc now
PET_WFDEI_200002.nc is complete
Processing celsius_Tair_WFDEI_201607.nc now
PET_WFDEI_201607.nc is complete
Processing celsius_Tair_WFDEI_200109.nc now
PET_WFDEI_200109.nc is complete
Processing celsius_Tair_WFDEI_199701.nc now
PET_WFDEI_199701.nc is complete
Processing celsius_Tair_WFDEI_198909.nc now
PET_WFDEI_198909.nc is complete
Processing celsius_Tair_WFDEI_200410.nc now
PET_WFDEI_200410.nc is complete
Processing celsius_Tair_WFDEI_199212.nc now
PET_WFDEI_199212.nc is complete
Processing celsius_Tair_WFDEI_199505.nc now
PET_WFDEI_199505.nc is complete
Processing celsius_Tair_WFDEI_198903.nc now
PET_WFDEI_198903.nc is complete
Processing celsius_Tair_WFDEI_200110.nc now
PET_WFDEI_200110.nc is complete
Processing celsius_Tair_WFDEI_198505.nc now
PET_WFDEI_19

PET_WFDEI_198611.nc is complete
Processing celsius_Tair_WFDEI_199903.nc now
PET_WFDEI_199903.nc is complete
Processing celsius_Tair_WFDEI_199406.nc now
PET_WFDEI_199406.nc is complete
Processing celsius_Tair_WFDEI_198806.nc now
PET_WFDEI_198806.nc is complete
Processing celsius_Tair_WFDEI_199204.nc now
PET_WFDEI_199204.nc is complete
Processing celsius_Tair_WFDEI_200511.nc now
PET_WFDEI_200511.nc is complete
Processing celsius_Tair_WFDEI_198801.nc now
PET_WFDEI_198801.nc is complete
Processing celsius_Tair_WFDEI_199209.nc now
PET_WFDEI_199209.nc is complete
Processing celsius_Tair_WFDEI_201208.nc now
PET_WFDEI_201208.nc is complete
Processing celsius_Tair_WFDEI_199004.nc now
PET_WFDEI_199004.nc is complete
Processing celsius_Tair_WFDEI_198606.nc now
PET_WFDEI_198606.nc is complete
Processing celsius_Tair_WFDEI_199605.nc now
PET_WFDEI_199605.nc is complete
Processing celsius_Tair_WFDEI_200107.nc now
PET_WFDEI_200107.nc is complete
Processing celsius_Tair_WFDEI_200801.nc now
PET_WFDEI_20

PET_WFDEI_199601.nc is complete
Processing celsius_Tair_WFDEI_198502.nc now
PET_WFDEI_198502.nc is complete
Processing celsius_Tair_WFDEI_198209.nc now
PET_WFDEI_198209.nc is complete
Processing celsius_Tair_WFDEI_198411.nc now
PET_WFDEI_198411.nc is complete
Processing celsius_Tair_WFDEI_200508.nc now
PET_WFDEI_200508.nc is complete
Processing celsius_Tair_WFDEI_199906.nc now
PET_WFDEI_199906.nc is complete
Processing celsius_Tair_WFDEI_201004.nc now
PET_WFDEI_201004.nc is complete
Processing celsius_Tair_WFDEI_199002.nc now
PET_WFDEI_199002.nc is complete
Processing celsius_Tair_WFDEI_199508.nc now
PET_WFDEI_199508.nc is complete
Processing celsius_Tair_WFDEI_198403.nc now
PET_WFDEI_198403.nc is complete
Processing celsius_Tair_WFDEI_200009.nc now
PET_WFDEI_200009.nc is complete
Processing celsius_Tair_WFDEI_200011.nc now
PET_WFDEI_200011.nc is complete
Processing celsius_Tair_WFDEI_200906.nc now
PET_WFDEI_200906.nc is complete
Processing celsius_Tair_WFDEI_199111.nc now
PET_WFDEI_19

PET_WFDEI_199802.nc is complete
Processing celsius_Tair_WFDEI_198804.nc now
PET_WFDEI_198804.nc is complete
Processing celsius_Tair_WFDEI_201204.nc now
PET_WFDEI_201204.nc is complete
Processing celsius_Tair_WFDEI_198512.nc now
PET_WFDEI_198512.nc is complete
Processing celsius_Tair_WFDEI_200812.nc now
PET_WFDEI_200812.nc is complete
Processing celsius_Tair_WFDEI_198404.nc now
PET_WFDEI_198404.nc is complete
Processing celsius_Tair_WFDEI_199008.nc now
PET_WFDEI_199008.nc is complete
Processing celsius_Tair_WFDEI_201504.nc now
PET_WFDEI_201504.nc is complete
Processing celsius_Tair_WFDEI_198305.nc now
PET_WFDEI_198305.nc is complete
Processing celsius_Tair_WFDEI_200612.nc now
PET_WFDEI_200612.nc is complete
Processing celsius_Tair_WFDEI_198506.nc now
PET_WFDEI_198506.nc is complete
Processing celsius_Tair_WFDEI_198907.nc now
PET_WFDEI_198907.nc is complete
Processing celsius_Tair_WFDEI_198906.nc now
PET_WFDEI_198906.nc is complete
Processing celsius_Tair_WFDEI_199606.nc now
PET_WFDEI_19